Naive Bayes model is the idea of using Bayes theorem (posterior probability = prior probability * likelihood) to estimate categorical outcomes;

The Naive bayes model assumes: ALL features included in the model are independent

In our project, we can use our knowledge about the news data to predict whether or not DJIA of a specific company increses or decreases based on naives model 

The first portion of the code describes some preprocessing of our data: i.e. tokenizing (breaking up sentences into smaller chunks), stemming and TD-IDF vectorizer (a type of word embedding, which is to sort words into vectors that have different weights). 

In [54]:
# Import necessary libraries 
import re
import nltk
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB # this is the library that assumes the likelihood function to have a multinomial distribution
from sklearn.metrics import accuracy_score

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer

english_stemmer=nltk.stem.SnowballStemmer('english')

In [39]:
# import the data; then split the data between testing and training sets based on date
df = pd.read_csv('../Data/Combined_News_DJIA.csv')
train = df[df['Date'] < '2015-01-01']
test = df[df['Date'] > '2014-12-31']


In [60]:
# Data cleaning 
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
string_train_head  = ''.join(trainheadlines)
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
string_test_head = ''.join(testheadlines)


In [61]:
len(trainheadlines)
len(testheadlines)

trainheadlines[0]
type(string_train_head)

str

In [63]:
# Eliminate stopwords
import nltk
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

train_token = word_tokenize(string_train_head)
test_token = word_tokenize(string_test_head)

train_filtered = []
for t in train_token:
    if t not in stopWords:
        train_filtered.append(t)

print(train_filtered[0:10]) # print an example wordlist without stopwords 

test_filtered = []
for d in test_token:
    if d not in stopWords:
        test_filtered.append(d)

print(test_filtered[0:10]) # print an example wordlist without stopwords 
    


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ruiyuzeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['b', "''", 'Georgia', "'downs", 'two', 'Russian', 'warplanes', "'", 'countries', 'move']
['Most', 'cases', 'cancer', 'result', 'sheer', 'bad', 'luck', 'rather', 'unhealthy', 'lifestyles']


In [64]:
# Stemming; the PorterStemmer does not seem to distinguish between verb and nouns
# for example, 'warplanes' becomes 'warplan' after applying the stemming function 
ps = PorterStemmer()
stemmed_train = []
stemmed_test = []

for train in train_filtered:
    stemmed_train.append(ps.stem(train))

for test in test_filtered:
    stemmed_test.append(ps.stem(test))
    
print(stemmed_train[0:10])
print(stemmed_test[0:10])

['b', "''", 'georgia', "'down", 'two', 'russian', 'warplan', "'", 'countri', 'move']
['most', 'case', 'cancer', 'result', 'sheer', 'bad', 'luck', 'rather', 'unhealthi', 'lifestyl']


In [52]:
# Word embedding for training and testing set 
tfidf = TfidfVectorizer(min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
tfidf_train = tfidf.fit_transform(trainheadlines) 
tfidf_test = tfidf.transform(testheadlines) # 
print(tfidf_train.shape)
print(tfidf_test.shape)

(1611, 529)
(378, 529)


In [55]:
advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(tfidf_train, train["Label"])
preds = advancedmodel.predict(tfidf_test)
acc=accuracy_score(test['Label'], preds)

In [56]:
acc # the accuracy score of the naive bayes model where no stemming and processing is applied to the training and testing set

0.5132275132275133

In [65]:
len(stemmed_train)

597245

In [66]:
len(stemmed_test)

136150

In [70]:
tfidf = TfidfVectorizer(min_df=0.01, max_df=1.0, max_features = 200000, ngram_range = (1, 1))
tfidf_train = tfidf.fit_transform(train_filtered) 
tfidf_test = tfidf.transform(test_filtered) # 
print(tfidf_train.shape)
print(tfidf_test.shape)

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.